# 1.1 Installations

First, you need to install `python3`:
* MacOS: http://docs.python-guide.org/en/latest/starting/install3/osx/
* Linux: http://docs.python-guide.org/en/latest/starting/install3/linux/
* Windows: https://www.python.org/downloads/windows/

Then, you need to make sure you installed `pip`:
```bash
python3 -m pip install pip
```
Assuming you installed `git`, you then do:
```bash
git clone https://github.com/cmu-mars/model-learner.git
cd model-learner
git checkout tutorial
python3 -m pip install --upgrade .
```
Now, you are ready to start!

# 1.2 Import libraries

In [1]:
# import the libraries, classes, methods we need for this tutorial
from learner.mlearner import MLearner
from learner.model import genModelTermsfromString, Model, genModelfromCoeff
import numpy as np

# 1.3 Define model

In [2]:
# Let's define a model, in this case a polynomial model
# Polynomial models are a great tool for determining which input factors drive responses and in what direction.
# Here we define a model with 20 dimensions, each variable represents a dimension and influence of each variable
# is different, e.g., o0 has the coefficient of 1, while o1 has the coefficient of 2 so its effect is twice comparing
# with o0. Also, we have two terms that represents the interactions of variables, e.g., 3 * o3 * o6.

ndim = 20 # defines the dimension of the model, i.e., the number of variables in the model
true_model = """10 + 1.00 * o0 + 2.00 * o1 + 3.00 * o2 +
4.00 * o3 + 5.00 * o4 + 6.00 * o5 + 7.00 * o6 + 8.00 * o7 + 
1.00 * o8 + 2.00 * o9 + 3.00 * o10 + 4.00 * o11 + 5.00 * o12 + 
6.00 * o13 + 7.00 * o14 + 8.00 * o15 + 1.00 * o16 + 2.00 * o17 + 
3.00 * o18 + 4.00 * o19 + 1 * o0 * o1 + 3 * o3 * o6"""

# 1.4 Build a model

In [3]:
# The model above is just a representation in string, so we need to build a model that we can evaluate given an input
power_model_terms = genModelTermsfromString(true_model)
true_power_model = Model(power_model_terms, ndim)
true_power_model.toString()

'1.00 * o0 + 2.00 * o1 + 3.00 * o2 + 4.00 * o3 + 5.00 * o4 + 6.00 * o5 + 7.00 * o6 + 8.00 * o7 + 1.00 * o8 + 2.00 * o9 + 3.00 * o10 + 4.00 * o11 + 5.00 * o12 + 6.00 * o13 + 7.00 * o14 + 8.00 * o15 + 1.00 * o16 + 2.00 * o17 + 3.00 * o18 + 4.00 * o19 + 1.00 * o0 * o1 + 3.00 * o3 * o6 + 10.0'

# 1.5 some example configurations:

In [33]:
# lets see how we evaluate the model with specific input
c0 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c1 = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c2 = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c3 = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c23 = [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# 1.6 some measurements

In [37]:
# let's try with array of zeros, what you expect to get as a result of evaluation? 10?
yc0 = true_power_model.evaluateModelFast(np.matrix(c0))
print(c0, yc0[0])
yc1 = true_power_model.evaluateModelFast(np.matrix(c1))
print(c1, yc1[0])
yc2 = true_power_model.evaluateModelFast(np.matrix(c2))
print(c2, yc2[0])
yc3 = true_power_model.evaluateModelFast(np.matrix(c3))
print(c3, yc3[0])
yc23 = true_power_model.evaluateModelFast(np.matrix(c23))
print(c23, yc23[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [[ 10.]]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [[ 11.]]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [[ 12.]]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [[ 13.]]
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [[ 15.]]


# 1.7 Sample data

In [38]:
# here we select how many sample data points we woould like to use for learning and sample configurations
budget = 10000
learner = MLearner(budget, ndim, true_power_model)
learner.sample_random()
learner.X, learner.y

(array([[1, 1, 1, ..., 1, 1, 1],
        [1, 0, 1, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 1, 1],
        ..., 
        [0, 0, 1, ..., 1, 1, 1],
        [0, 0, 1, ..., 1, 0, 1],
        [0, 0, 1, ..., 1, 0, 1]]),
 array([ 64.,  72.,  58., ...,  63.,  50.,  44.]))

# 1.8 Learn the model without interaction terms

In [39]:
learned_model_without_interactions = learner.learn_without_interactions()
learned_model_terms = genModelfromCoeff(learned_model_without_interactions.named_steps['linear'].coef_, ndim)
learned_model = Model(learned_model_terms, ndim)
learned_model.toString()

'1.52 * o0 + 2.50 * o1 + 3.00 * o2 + 5.49 * o3 + 5.01 * o4 + 5.98 * o5 + 8.48 * o6 + 7.98 * o7 + 1.00 * o8 + 2.02 * o9 + 2.98 * o10 + 4.00 * o11 + 4.98 * o12 + 5.99 * o13 + 7.01 * o14 + 8.00 * o15 + 1.02 * o16 + 2.02 * o17 + 2.99 * o18 + 4.01 * o19'

# 1.9 Learn the model with pairwise interaction terms

In [40]:
# lets start learning
learned_model_with_interactions = learner.learn_with_interactions()
learned_model_terms = genModelfromCoeff(learned_model_with_interactions.named_steps['linear'].coef_, ndim)
learned_model = Model(learned_model_terms, ndim)
learned_model.toString()

'1.00 * o0 + 2.00 * o1 + 3.00 * o2 + 4.00 * o3 + 5.00 * o4 + 6.00 * o5 + 7.00 * o6 + 8.00 * o7 + 1.00 * o8 + 2.00 * o9 + 3.00 * o10 + 4.00 * o11 + 5.00 * o12 + 6.00 * o13 + 7.00 * o14 + 8.00 * o15 + 1.00 * o16 + 2.00 * o17 + 3.00 * o18 + 4.00 * o19 + -0.00 * o0 * o1 + -0.00 * o0 * o2 + -0.00 * o0 * o3 + 0.00 * o0 * o4 + 0.00 * o0 * o5 + 0.00 * o0 * o6 + -0.00 * o0 * o7 + -0.00 * o0 * o8 + 0.00 * o0 * o9 + 0.00 * o0 * o10 + 0.00 * o0 * o11 + -0.00 * o0 * o12 + 0.00 * o0 * o13 + -0.00 * o0 * o14 + -0.00 * o0 * o15 + 0.00 * o0 * o16 + -0.00 * o0 * o17 + 0.00 * o0 * o18 + 0.00 * o0 * o19 + -0.00 * o1 * o2 + 0.00 * o1 * o3 + 0.00 * o1 * o4 + 0.00 * o1 * o5 + -0.00 * o1 * o6 + -0.00 * o1 * o7 + 0.00 * o1 * o8 + 0.00 * o1 * o9 + 0.00 * o1 * o10 + -0.00 * o1 * o11 + 0.00 * o1 * o12 + -0.00 * o1 * o13 + -0.00 * o1 * o14 + 0.00 * o1 * o15 + -0.00 * o1 * o16 + 0.00 * o1 * o17 + 0.00 * o1 * o18 + 0.00 * o1 * o19 + 0.00 * o2 * o3 + 0.00 * o2 * o4 + -0.00 * o2 * o5 + -0.00 * o2 * o6 + 0.00 * o2 * o7